In [55]:
from pathlib import Path
from typing import List

from os import path
import numpy as np
from skimage import io,util,filters
from skimage.transform import EuclideanTransform, warp
from scipy import ndimage

from pystackreg import StackReg
# from twophotonUtils import parse_unregistered_channels
# from twophotonUtils import find_most_likely_z_slice_using_CC, z_translate_and_pad
# from imageLoadingWidgets import LoadTimepointForInspection

from os import path
from glob import glob
from re import findall
from tqdm import tqdm

import stackview

# Extract the first ome.tiff file from every subfolder, load, then separate the two channels
def sort_by_slice(filename):
    z = findall('_(\d+).ome.tif',filename)[0]
    return int(z)

def read_prairie_ome_xml(header_ome,channel_names,registerZ=False,reference_idx=0):
    import warnings
    warnings.filterwarnings("ignore")
    
    reference = channel_names[reference_idx]

    d = path.split(path.dirname(header_ome))[0]
    if np.all([path.exists(path.join(d,f'{chan}_reg.tif')) for chan in channel_names]):
        print(f'All *_reg channels exist for {channel_names} channels')
        return
    
    # Load ome-tif
    print(f'Loading {d}')
    stack = io.imread(header_ome,is_ome=True)
    
    if stack.ndim > 3:
        images = [im for im in stack]
    else:
        images = [stack]

    if registerZ and reference is not None:
        # Use StackReg
        print(f'Registering {d}')
        sr = StackReg(StackReg.TRANSLATION) # There should only be slight sliding motion within a single stack
        T = sr.register_stack(B,reference='previous',n_frames=20,axis=0) #Obtain the transformation matrices
        for i,im in enumerate(images):
            images[i] = sr.transform_stack(im,tmats=T) # Apply to all channels

    for i,im in enumerate(images):
        output_path = path.join( d,f'{channel_names[i]}_reg.tif')
        io.imsave(output_path,util.img_as_uint(im/im.max()),check_contrast=False)
    
    print(f'Saved with {output_path}')

    return images


# 0. Get file paths

Step one: Load all the OME-TIFFs and re-save as multipage TIFFs for each time point in region
Will perform a StackReg on each stack just in case there is small movement.


In [65]:
# dirname = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/09-29-2025 Ear palbo pilot/M1tail 3017 H2B FUCCI/Right ear/4. Day 4/R2'
# dirname = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Dermal topography/10-03-2025 Cdh5-RFP blood vessels/Male 6121/Left paw R2/'
dirname = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen\'s files - 11-01-2025 H2B FUCCI Palbo/M1 cage 5971/Left ear/'

subfolders = glob(path.join(dirname,'**/*/ZSeries*/'))

header_ome_h2b = []
header_ome_fucci = []

for d in subfolders:
    
    ome_tifs = glob(path.join(d,'*.ome.tif'))
    ome_tifs = sorted(ome_tifs) # Sort by channel #
    ome_tifs = sorted(ome_tifs, key = sort_by_slice) # Sort by slice #
    if len(ome_tifs) < 30:
        print(f'Skipping {d}')
    else:
        if len(findall('1020nm',path.split(path.split(d)[0])[1])) == 0:
            header_ome_h2b.append(ome_tifs[0])
        else:
            header_ome_fucci.append(ome_tifs[0])

print(f'Found {len(header_ome_h2b)} raw B/G stacks and {len(header_ome_fucci)} raw R/R_shg stacks')

Skipping /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen's files - 11-01-2025 H2B FUCCI Palbo/M1 cage 5971/Left ear/R1/Overview/ZSeries-11012025-1632-2437/
Skipping /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen's files - 11-01-2025 H2B FUCCI Palbo/M1 cage 5971/Left ear/R1/Overview/ZSeries-11012025-1632-2436/
Skipping /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen's files - 11-01-2025 H2B FUCCI Palbo/M1 cage 5971/Left ear/R3/Overview/ZSeries-11042025-1510-2474/
Skipping /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen's files - 11-01-2025 H2B FUCCI Palbo/M1 cage 5971/Left ear/R3/Overview/ZSeries-11042025-1510-2473/
Skipping /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence

In [68]:
# Parse multiple ZSeries within the same subfolder
from collections import defaultdict

region_timepoints = defaultdict(list)
for f in header_ome_h2b:
    region_timepoints[ path.dirname(path.dirname(f)) ].append(f)
# Go through each timepoint and pick the 'latest' (aka last-sorted) stack
final_h2b_timepoints = []
for list_of_f in region_timepoints.values():
    list_of_f = sorted(list_of_f) # Rely on default sortings
    final_h2b_timepoints.append(list_of_f[-1])

region_timepoints = defaultdict(list)
for f in header_ome_fucci:
    region_timepoints[ path.dirname(path.dirname(f)) ].append(f)
# Go through each timepoint and pick the 'latest' (aka last-sorted) stack
final_fucci_timepoints = []
for list_of_f in region_timepoints.values():
    list_of_f = sorted(list_of_f) # Rely on default sortings
    final_fucci_timepoints.append(list_of_f[-1])

print(f'Final number of B/G channels: {len(final_h2b_timepoints)}')
print(f'Final number of R/R_shg channels: {len(final_fucci_timepoints)}')

Final number of B/G channels: 8
Final number of R/R_shg channels: 8


["/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen's files - 11-01-2025 H2B FUCCI Palbo/M1 cage 5971/Left ear/R1/1. Day 1/ZSeries-11022025-1530_1020nm-2459/ZSeries-11022025-1530_1020nm-2459_Cycle00001_Ch2_000001.ome.tif",
 "/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen's files - 11-01-2025 H2B FUCCI Palbo/M1 cage 5971/Left ear/R1/0. Day 0/ZSeries-11012025-1632_1020nm-2439/ZSeries-11012025-1632_1020nm-2439_Cycle00001_Ch2_000001.ome.tif",
 "/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen's files - 11-01-2025 H2B FUCCI Palbo/M1 cage 5971/Left ear/R3/4. Day 4/ZSeries-11052025-1523_1020nm-2485/ZSeries-11052025-1523_1020nm-2485_Cycle00001_Ch2_000001.ome.tif",
 "/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen's files

# 1. Load each OME-XML and resave

In [71]:
registerZ = False # Perform an 'internal' stackreg?

# Load B/G (H2B)
for header_h2b in final_h2b_timepoints:
    _ = read_prairie_ome_xml(header_h2b,channel_names = ['G','B'], reference_idx=0, registerZ=registerZ)

# Load B/G (H2B)
for header_fucci in final_fucci_timepoints:
    _ = read_prairie_ome_xml(header_fucci,channel_names = ['R','R_shg'], reference_idx=0, registerZ=registerZ)


<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised

All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
All *_reg channels exist
Loading /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen's files - 11-01-2025 H2B FUCCI Palbo/M1 cage 5971/Left ear/R2/3. Day 3


<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised

Saved with /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/Julie Charlotte Nielsen's files - 11-01-2025 H2B FUCCI Palbo/M1 cage 5971/Left ear/R2/3. Day 3/R_shg_reg.tif
